# Inferencia Estadística
## Tarea 1

### Autor:
- [Sergio García Prado](https://garciparedes.me)

In [1]:
rm(list = ls())

In [2]:
library(magrittr)
library(IRdisplay, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(Bhat)

# Se lanzan seis monedas en cien ocasiones y se anota el número de caras en cada lanzamiento. Los resultados fueron:

In [3]:
coins <- data.frame(tosses = c(0, 1, 2, 3, 4, 5, 6),
                    freq = c(2, 8, 10, 12, 16, 30, 22))

In [4]:
coins %>% 
    set_colnames(c('Número de caras', 'Frecuencias')) %>% 
    t() %>% 
    display()

Número de caras,0,1,2,3,4,5,6
Frecuencias,2,8,10,12,16,30,22


## Obtener el $p‐valor$ del test de razón de verosimilitud para contrastar la siguiente hipótesis:
$$H_0: \text{Todas las monedas tienen la misma probabilidad de cara}$$

Vamos a definir las siguientes variables aleatorias: 

$$Y_1,..., Y_6 \ iid \quad | \ Y_i \sim Bin(n, p_i)$$

Donde $n = 100$ es el número de realizaciones de la muestra, tal y como se indica en el enunciado y la variable $Y_i$ representa el *número de caras obtenidas por la moneda i-ésima*. Por tanto, el contraste se puede reescribir utilizando esta notación de tal manera que la hipótesis sea:
$$H_0: p_i = p_j \quad \forall i, j \in \{1,..., 6\}$$

Sin embargo, esto no es lo que se proporciona en la muestra obtenida

In [5]:
n <- coins %>%
    summarise((max(tosses) * sum(freq))) %>%
    pull()

In [6]:
display_latex(paste0('$$n = ', n, '$$'))

In [7]:
p.hat <- coins %>%
    mutate(total = freq * tosses) %>%
    summarise(sum(total) / n) %>%
    pull()

In [8]:
display_latex(paste0('$$\\hat{p} = ', round(p.hat, digits = 4), '$$'))

In [9]:
coins <- coins %>%
    mutate(freq.rel = freq / sum(freq))

In [10]:
coins %>% 
    set_colnames(c('Número de caras', 'Frecuencias', 'Frecuencias Relativas')) %>% 
    t() %>% 
    round(digits = 4) %>%
    display()

Número de caras,0.00,1.00,2.0,3.00,4.00,5.0,6.00
Frecuencias,2.00,8.00,10.0,12.00,16.00,30.0,22.00
Frecuencias Relativas,0.02,0.08,0.1,0.12,0.16,0.3,0.22


In [11]:
coins <- coins %>%
    mutate(expected.freq.rel = dbinom(0:6, 6, p.hat))

In [12]:
coins %>% 
    set_colnames(c('Número de caras', 'Frecuencias', 'Frecuencias Relativas',
                   'Frecuencias Relativas Esperadas')) %>% 
    t() %>% 
    round(digits = 4) %>%
    display()

Número de caras,0.000,1.0000,2.0000,3.0000,4.000,5.0000,6.0000
Frecuencias,2.000,8.0000,10.0000,12.0000,16.000,30.0000,22.0000
Frecuencias Relativas,0.020,0.0800,0.1000,0.1200,0.160,0.3000,0.2200
Frecuencias Relativas Esperadas,0.001,0.0131,0.0704,0.2026,0.328,0.2831,0.1018


In [13]:
G <- coins %>%
    summarise(2 * sum(expected.freq.rel * log(expected.freq.rel / freq.rel))) %>%
    pull()

In [14]:
display_latex(paste0('$$G = ', round(G, 4), '$$'))

In [15]:
pvalue <- pchisq(G, df=6)

In [16]:
display_latex(paste0('$$p-valor = ', round(pvalue, 4), '$$'))

## En el modelo que define la hipótesis nula obtener intervalos de confianza ($95\%$) para el parámetro, basados en los estadísticos de *Wald (W)* y de *razón de verosimilitud (VR)*.

In [17]:
alpha <- 0.05

Cálculo por ecuación explícita

In [18]:
W.var <- p.hat * (1 - p.hat) / n
W.IC <- p.hat + c(-1, 1) * qnorm(1 - alpha / 2) * sqrt(W.var)

In [19]:
display_latex(paste0('$$\\left(', round(W.IC[1], 5), ', ', round(W.IC[2], 5) , '\\right)$$'))

Cálculo por optimización numérica

In [20]:
nloglhood <- function(p, n = 600, y = 410) {
    return( -(log(choose(n, y)) + y * log(p) + (n - y) *log(1 - p)) )
}

In [21]:
opt <- optim(0.5, nloglhood,lower = 0.0001,upper = 0.9999, 
             hessian = TRUE, method = "L-BFGS-B")
phat <- opt$par
phat.var <- as.numeric(1 / opt$hessian)

In [22]:
phat + c(-1, 1) * qnorm(1 - alpha / 2) * sqrt(phat.var)

[1] 0.6461118 0.7205537

In [23]:
control.list=list(label="p",est=p.hat,low=0,upp=1)
invisible(capture.output(LR.ci <- plkhci(control.list, nloglhood, "p")))

In [24]:
LR.ci

[1] 0.6453866 0.7197188

# Considerar el vector aleatorio $X = (X_1, ..., X_5)$ con distribución multinomial, tal que $p = \left(\frac{1}{2}, \frac{\theta}{4}, \frac{1 - \theta}{4}, \frac{1 - \theta}{4}, \frac{\theta}{4}\right)$

Si el valor observado de $Y$ es $y = \left(125, 18, 20, 34\right)$, usar $3$ iteraciones del *algoritmo EM* para aproximar el estimador máximo verosímil de $\theta$, partiendo del valor inicial $\theta^{(0)} = 0.5$.

In [25]:
# TODO